In [ ]:
!nvidia-smi

Wed Aug 17 09:14:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports

In [ ]:
%%capture
!pip install transformers
!pip install pytorch-lightning

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import re

import random
import torch

def set_seed(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

import warnings
warnings.simplefilter('ignore')

import gc

In [ ]:
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/signate_student_cup_2022
train_path = './data/train.csv'
test_path = './data/test.csv'
submit_path = './data/submit_sample.csv'

/content/drive/MyDrive/signate_student_cup_2022


In [ ]:
job_flags = ['Data scientist', 'Machine learning engineer','Software engineer','Consultant']

In [ ]:
def text_cleaning(texts):
    clean_texts = []
    num_lines = []
    num_words = []
    words_chunk = []
    for text in texts:
        clean_lines = []
        # print(text, "\n")
        lines = text.split(r"</li>")
        for line in lines:
            line = remove_tag(line)
            # print(line)
            #バックスラッシュをスペースに置き換え
            clean_line = re.sub(r'[\\]', '', line)
            # clean_line = re.sub(r'[/]', ' and ', line)# test
            # print(clean_line)
            clean_line = clean_line.strip()
            # print(clean_line, "\n")
            clean_line = clean_line + ('' if clean_line.endswith('.') else '.')
            if len(clean_line)!=1:
                # print(clean_line, "\n")
                clean_lines.append(clean_line)
        # print(clean_lines)
        clean_texts.append(' '.join(clean_lines))
        
        num_lines.append(len(clean_lines))
        num_word = len(str(clean_lines).split())
        num_words.append(num_word)
        word_chunk = ((num_word-1)//240)+1
        words_chunk.append(word_chunk)

    return clean_texts, num_lines, num_words, words_chunk

def remove_tag(x):
    p = re.compile(r"<[^>]*?>")
    return p.sub('',x)

In [ ]:
train_data = pd.read_csv(train_path)
train_data['description'], num_lines, num_words, words_chunk = text_cleaning(train_data['description'])
train_data['num_lines'] = num_lines
train_data['num_words'] = num_words
train_data['words_chunk'] = words_chunk
train_data['jobflag'] = train_data['jobflag']-1
train_data.head(20)

,id,description,jobflag,num_lines,num_words,words_chunk
0,0,Develop cutting-edge web applications that per...,2,8,112,1
1,1,"Designs and develops high quality, scalable an...",2,15,221,1
2,2,Functions as a point person for Network Strate...,3,9,190,1
3,3,"Work on the technical design, development, rel...",2,5,79,1
4,4,Quantify the resources required for a task/pro...,3,2,33,1
5,5,Participates in standard business and technica...,2,4,49,1
6,6,"Create project plans, establish timelines, and...",3,5,85,1
7,7,"Facilitate pre-sales initiatives, such as live...",3,11,105,1
8,8,Consolidate dashboards across the team and hel...,0,1,17,1
9,9,Maintain and improve existing predictive model...,0,4,53,1


In [ ]:
test_data = pd.read_csv(test_path)
test_data['description'], num_lines, num_words, words_chunk = text_cleaning(test_data['description'])
test_data['num_lines'] = num_lines
test_data['num_words'] = num_words
test_data['words_chunk'] = words_chunk
test_data.head(5)

,id,description,num_lines,num_words,words_chunk
0,1516,Building decision-making models and proposing ...,2,30,1
1,1517,Educate homeowners on the benefits of solar en...,5,43,1
2,1518,"Design, develop, document, and implement web a...",8,82,1
3,1519,Apply advanced technical expertise and skills ...,6,81,1
4,1520,Project manage and deliver against our roadmap...,4,35,1


In [ ]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 410 kB 17.3 MB/s 


In [ ]:
from transformers import set_seed
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

"""
augs = {}
augs["synonym_replace"] = naw.SynonymAug(aug_src='wordnet')
augs["random_insert"] = naw.ContextualWordEmbsAug(model_path="distilbert-base-uncased", 
                                device="cpu", action="insert", aug_max=1)
augs["random_swap"] = naw.RandomWordAug(action="swap")
augs["random_delete"] = naw.RandomWordAug()
augs["bt_en_de"] = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-en-de', 
    to_model_name='facebook/wmt19-de-en'
)
"""

aug = naw.ContextualWordEmbsAug(model_path="distilbert-base-uncased", 
                                device="cuda", action="substitute", aug_p=0.5)

def augmentation(df, aug_num):
    aug_samples = []

    for _ in range(aug_num):
        target_sample = df.sample()
        # print(f'Original Text >> {target_sample["description"].values[0]}')
        aug_desc = aug.augment(target_sample["description"].values[0])[0]
        # print(f'Augmented Text >> {aug_desc} \n')
        aug_samples += [[aug_desc, target_sample["jobflag"].values[0]]]

    aug_df = pd.DataFrame(aug_samples, columns=["description", "jobflag"])
    aug_df = pd.concat([df, aug_df]).reset_index(drop=True)
    return aug_df

"""new_train = augmentation(train_data[["description", "jobflag"]], 50)
print(train_data.shape)
print(new_train.shape)"""

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


'new_train = augmentation(train_data[["description", "jobflag"]], 50)\nprint(train_data.shape)\nprint(new_train.shape)'

In [ ]:
def text_augmentation(df, augmentation_weight):
    """
    df: pandas dataframe
    augmentation_weight: Percentage of more data per label  ex) [1.0, 5.0, 1.0, 1.0] 
    """

    for label in range(4):
        temp_df = df[df['jobflag']==label][["description", "jobflag"]]
        aug_num = int(len(temp_df)*(augmentation_weight[label]-1.0))
        print(f'Label {label} >> Aug num {aug_num}')
        print(temp_df.shape)

        if aug_num > 0:# augmentation
            temp_df = augmentation(temp_df, aug_num)
        print(temp_df.shape)

        if label==0:
            augmented_df = temp_df
        else:
            augmented_df = pd.concat([augmented_df, temp_df])

    augmented_df = augmented_df.sample(frac=1, ignore_index=True).drop_duplicates().reset_index(drop=True)
    return augmented_df


augmented_df = text_augmentation(train_data, [1.0, 5.0, 1.0, 1.0])

Label 0 >> Aug num 0
(468, 2)
(468, 2)
Label 1 >> Aug num 352
(88, 2)
(440, 2)
Label 2 >> Aug num 0
(455, 2)
(455, 2)
Label 3 >> Aug num 0
(505, 2)
(505, 2)


In [ ]:
augmented_df["jobflag"].value_counts()

3    494
0    458
2    453
1    438
Name: jobflag, dtype: int64

In [ ]:
augmented_df.drop_duplicates()["jobflag"].value_counts()

3    494
0    458
2    453
1    438
Name: jobflag, dtype: int64